# Track and Analyze GitHub Star Growth With Kandy and Kotlin DataFrame

[Kotlin DataFrame](https://kotlin.github.io/dataframe/overview.html) and [Kandy](https://kotlin.github.io/kandy/welcome.html) are two powerful tools for data analysis in Kotlin. Kotlin DataFrame simplifies data manipulation and processing, while Kandy allows you to create stunning visualizations directly within your Kotlin projects.

In this post, we’ll show you how these tools can be used together within [Kotlin Notebook](https://kotlinlang.org/docs/kotlin-notebook-overview.html) to analyze the star history of GitHub repositories. This isn't just a simple exercise for demonstration purposes – it's a tutorial that can help you learn how to analyze your own repositories, understand their popularity trends, and visualize your data effectively.

## **Why analyze star history?**

Understanding the star history of a GitHub repository can provide insights into its popularity and growth over time. By analyzing this data, you can see how different events and activities impact the interest in your project. Our goal is to equip you with the knowledge and tools to perform this analysis on your own repositories.

In [11]:
// import of necessary libraries of current versions 
%useLatestDescriptors
// use ktor & serialization to get the data
%use ktor-client, serialization
// use dataframe to process data and kandy to visualize it
%use dataframe, kandy, datetime

## Obtaining repository stargazers data from GitHub

First, we need to gather data about the users who starred a given repository. To achieve this, we’ll use the [GitHub GraphQL API](https://docs.github.com/en/graphql), which requires a [GitHub access token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens). Here’s a simple function to request data about repo stars, including the starring time and user login:

In [12]:
import io.ktor.client.request.*
import io.ktor.http.*


/**
 *  We need to specify the repository owner and name, as well as the access token.
 * There can be up to 100 results on one response page.
 * For this example, we'll take only the first 3 results.
 * `endCursor` points to the end of the previous page (`null` for the first one).
 */
fun fetchStarHistoryPage(owner: String, name: String, token: String, first: Int = 100, endCursor: String? = null): NotebookHttpResponse {
    // GraphQL query
    val query = """
       query {
         repository(owner: "$owner", name: "$name") {
           stargazers(first: $first, after: $endCursor) {
             edges {
               starredAt
               node {
                 login
               }
             }
             pageInfo {
               endCursor
               hasNextPage
             }
           }
         }
       }
   """.trimIndent()
    // `http` is the default Ktor `HttpClient` for Notebook;
    // it has the same methods but without `suspend` modifiers, 
    // allowing you to make HTTP requests quickly and easily. 
    // Make a "post" request to the API with this query

    return http.post("https://api.github.com/graphql") {
        // Set authorization header with token
        bearerAuth(token)
        // Set content type header
        contentType(ContentType.Application.Json)
        // Set query as body
        setBody(buildJsonObject { put("query", query) })
    }
}

In [13]:
val ownerKotlin = "Kotlin"
val repoKandy = "kandy"
// Keep your token safe, as an environment variable or a system property!
// For example, you can place it in environment variables in Kotlin Notebook settings.
//val token = System.getenv("GITHUB_TOKEN")
val token = "YOUR_GITHUB_TOKEN"

A convenient and easy way to set an environment variable is through the Kotlin Notebook settings:

![kotlin_notebook_settings](https://github.com/user-attachments/assets/f52c9394-5427-4426-971f-16753fdf2da7)

To start, let's query a single page with a few users to examine the data.

In [14]:
val rawResponse = fetchStarHistoryPage(ownerKotlin, repoKandy, token, first = 3)
rawResponse

HttpResponse[https://api.github.com/graphql, 200 OK]

Next, we'll deserialize the JSON response to a Kotlin data class using the .deserializeJson() extension provided by our Kotlin Notebook Ktor integration. This makes it easier to work with the response body data in Kotlin.

In [15]:
val starHistorySimplePage = rawResponse.deserializeJson()
// Take the JSON string for further work with DataFrame
val responseAsJson = starHistorySimplePage.jsonString
starHistorySimplePage

```json
{
    "data": {
        "repository": {
            "stargazers": {
                "edges": [
                    {
                        "starredAt": "2022-07-13T22:46:16Z",
                        "node": {
                            "login": "manojselvam"
                        }
                    },
                    {
                        "starredAt": "2022-11-05T14:21:10Z",
                        "node": {
                            "login": "PoslavskySV"
                        }
                    },
                    {
                        "starredAt": "2022-11-05T18:42:37Z",
                        "node": {
                            "login": "dbolotin"
                        }
                    }
                ],
                "pageInfo": {
                    "endCursor": "Y3Vyc29yOnYyOpK5MjAyMi0xMS0wNVQxOTo0MjozNyswMTowMM4V4kpZ",
                    "hasNextPage": true
                }
            }
        }
    }
}
```

After executing the cell above, starHistorySimplePage is converted to a data class, allowing us to easily access those of its properties that correspond to JSON fields. This seamless integration with IDEA autocompletion makes working with the response straightforward.

![Serialized response completion](https://github.com/user-attachments/assets/863155dd-7f53-405a-a09d-cd96bf8ef83c)

For example, we can extract all the starring times from the page:

In [16]:
starHistorySimplePage.data.repository.stargazers.edges.map { it.starredAt }

[2022-07-13T22:46:16Z, 2022-11-05T14:21:10Z, 2022-11-05T18:42:37Z]

Next, let's parse the page data into a DataFrame. 

In [17]:
val starHistoryPageDF = DataFrame.readJsonStr(responseAsJson)
starHistoryPageDF

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-385875968"></table>

<p class="dataframe_description">DataFrame: rowsCount = 1, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }

We need two columns: one showing the user logins and the other their starring times. We can retrieve these columns as follows:

In [18]:
starHistoryPageDF.data.repository.stargazers.edges
    .single() // `edges` colum  contains single DataFrame with current page stargazers
    .flatten() // `login` is a subcolumn of `node`, after `flatten()` it is a simple column

starredAt,login
2022-07-13T22:46:16Z,manojselvam
2022-11-05T14:21:10Z,PoslavskySV
2022-11-05T18:42:37Z,dbolotin


Additionally, we need page meta-information, including whether there is a next page and the current page end cursor.

In [19]:
with(starHistoryPageDF.data.repository.stargazers.pageInfo) {
    // Both are columns with a single value
    println("end cursor: ${endCursor.single()}")
    println("has next page: ${hasNextPage.single()}")
}

end cursor: Y3Vyc29yOnYyOpK5MjAyMi0xMS0wNVQxOTo0MjozNyswMTowMM4V4kpZ
has next page: true


Now, let's create a function that iteratively processes all pages with stargazers and returns a DataFrame with complete information:

In [20]:
// Casts DataFrame to the type of a given DataFrame so we can use
// extension columns that have already been generated.
// Temporary workaround, will be available in future DataFrame releases
// (https://github.com/Kotlin/dataframe/pull/747)
inline fun <reified T> AnyFrame.castTo(df: DataFrame<T>): DataFrame<T> {
    return cast<T>(verify = true)
}

In [21]:
import io.ktor.client.statement.*


// Provide repo owner, name, and access token
fun fetchStarHistory(owner: String, name: String, token: String): AnyFrame {
    var hasNextPage: Boolean = true
    var endCursor: String? = null
    var buffer: DataFrame<*> = DataFrame.Empty
    while (hasNextPage) {
        val response = fetchStarHistoryPage(owner, name, token, 100, endCursor)
        // Cast type of DataFrame to the type of `starHistoryPageDF`,
        // so we can use its already-generated extensions
        val responseDF = DataFrame.readJsonStr(response.bodyAsText()).castTo(starHistoryPageDF)
        val stargazers = responseDF.data.repository.stargazers
        buffer = buffer.concat(stargazers.edges.first().flatten())
        val pageInfo = stargazers.pageInfo
        endCursor = "\"${pageInfo.endCursor.single()}\""
        hasNextPage = pageInfo.hasNextPage.single()
    }
    return buffer
}

Using this function, we can now retrieve all the Kandy stargazers:

In [22]:
val kandyStargazers = fetchStarHistory(ownerKotlin, repoKandy, token)
kandyStargazers

starredAt,login
2022-07-13T22:46:16Z,manojselvam
2022-11-05T14:21:10Z,PoslavskySV
2022-11-05T18:42:37Z,dbolotin
2022-12-10T00:30:59Z,cheroliv
2023-01-15T04:06:50Z,sureshg
2023-01-15T08:50:31Z,whyoleg
2023-01-22T11:39:05Z,JohnTurkson
2023-01-26T16:28:49Z,alexandrepiveteau
2023-01-29T07:14:13Z,xiang23
2023-02-02T15:01:55Z,shalaga44


Look at the DataFrame summary using the .describe() method, which shows meta-information and accumulated statistics about DataFrame columns:

In [23]:
kandyStargazers.describe()

name,type,count,unique,nulls,top,freq,min,p25,median,p75,max
starredAt,String,681,681,0,2022-07-13T22:46:16Z,1,2022-07-13T22:46:16Z,2023-12-14T18:11:13Z,2024-01-06T08:11:11Z,2024-06-20T15:01:24Z,2025-06-28T04:27:16Z
login,String,681,681,0,manojselvam,1,0xffrom,TarasZmyi,garretyoder,oxyroid,zsqw123


All login values are unique, indicating that the dataset is correct. Additionally, there are no null values, so no further processing is needed.

## Creating a DataFrame for cumulative star count analysis

We now have two key pieces of information: user logins and the times they award stars. Our next step is to perform an initial analysis.

We'll create a visualization showing the cumulative number of stars received over time, illustrating how user interest in our library grows and changes.

This approach will help us understand the dynamics of user engagement and the popularity of our library.

Here's how to transform this data:

1. Convert the `starredAt` column to `LocalDateTime`.

2. Sort the DataFrame by `starredAt`, in ascending order.

3. Add a `starsCount` column to track the total number of stars over time.

Put the processing code into a function so that it can be reused later on.

In [24]:
fun AnyFrame.processStargazers(): AnyFrame {
    return castTo(kandyStargazers)
        // Convert `starredAt` column to `LocalDateTime`
        .convert { starredAt }.toLocalDateTime()
        // Sort rows by `starredAt`
        .sortBy { starredAt }
        // Add `starsCount` column with total stars count at each row.
        // The star count is simply the row index increased by 1
        .add("starsCount") { index() + 1 }
}

In [25]:
val kandyStarHistory = kandyStargazers.processStargazers()
kandyStarHistory

starredAt,login,starsCount
2022-07-13T22:46:16,manojselvam,1
2022-11-05T14:21:10,PoslavskySV,2
2022-11-05T18:42:37,dbolotin,3
2022-12-10T00:30:59,cheroliv,4
2023-01-15T04:06:50,sureshg,5
2023-01-15T08:50:31,whyoleg,6
2023-01-22T11:39:05,JohnTurkson,7
2023-01-26T16:28:49,alexandrepiveteau,8
2023-01-29T07:14:13,xiang23,9
2023-02-02T15:01:55,shalaga44,10


## Visualizing star history: plotting with Kandy

With the data processed, we can now visualize the star history using Kandy. Here’s a simple line plot to show how the number of stars has changed over time. 

In [26]:
kandyStarHistory.plot {
    line {
        // The starring time corresponds to the `x` axis
        x(starredAt) {
            axis {
                // Set the name for the `x` axis
                name = "date"
                // Set the format for axis breaks
                breaks(format = "%b, %Y")
            }
        }
        // The stars count corresponds to the `y` axis
        y(starsCount) {
            // Set the name for the `y` axis
            axis.name = "GitHub stars"
        }
    }
    layout {
        title = "Kandy GitHub star history"
        size = 800 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UojL4t"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Kandy GitHub star history"
},
"mapping":{
},
"data":{
"starredAt":[1.657752376E12,1.66765807E12,1.667673757E12,1.670632259E12,1.67375561E12,1.673772631E12,1.674387545E12,1.674750529E12,1.674976453E12,1.675350115E12,1.675408744E12,1.675588285E12,1.675674089E12,1.675680685E12,1.675715159E12,1.67573036E12,1.675748333E12,1.6757488E12,1.675749329E12,1.675751863E12,1.675758317E12,1.67578179E12,1.675861417E12,1.675861829E12,1.675913952E12,1.675922947E12,1.675948684E12,1.675958048E12,1.676047476E12,1.676048333E12,1.676391105E12,1.676496362E12,1.676597958E12,1.676948714E12,1.677378223E12,1.67897314E12,1.679221344E12,1.679700779E12,1.6799184E12,1.68036127E12,1.681047606E12,1.681327305E12,1.6814385E12,1.68143885E12,1.681590162E12,1.681652693E12,1.681824155E12,1.682201702E12,1.682251358E12,1.682276556E12,1.68293626E12,1.683033915E12,1.683047937E12,1.683885575E12,1.683985043E12,1.685620132E12,1.68585262E12,1.686015721E12,1.686098215E12,1.68650696E12,1.68738686E12,1.687491503E12,1.687681586E12,1.688399934E12,1.688521939E12,1.688582287E12,1.688646578E12,1.688669174E12,1.689026092E12,1.689282982E12,1.689341997E12,1.689382129E12,1.6899506E12,1.689953276E12,1.689955396E12,1.689957355E12,1.689969537E12,1.689994317E12,1.690021986E12,1.690022974E12,1.690049489E12,1.69016283E12,1.690162892E12,1.690563562E12,1.690741188E12,1.690779996E12,1.691125019E12,1.691150646E12,1.691652159E12,1.691916388E12,1.692150284E12,1.692236017E12,1.692947778E12,1.692957081E12,1.693225707E12,1.693399671E12,1.693665307E12,1.694511733E12,1.694824385E12,1.694837018E12,1.694944123E12,1.696683456E12,1.696907638E12,1.697100579E12,1.697811722E12,1.698252394E12,1.698262479E12,1.698262841E12,1.698305842E12,1.698309951E12,1.698324538E12,1.698357092E12,1.698366E12,1.698368801E12,1.698374102E12,1.698377327E12,1.698378851E12,1.698378863E12,1.698387957E12,1.698388163E12,1.698388984E12,1.698392645E12,1.698394632E12,1.698397319E12,1.698397404E12,1.698401725E12,1.698402252E12,1.698411376E12,1.698416804E12,1.698424836E12,1.698670733E12,1.698677571E12,1.698683248E12,1.698743417E12,1.698744643E12,1.698876002E12,1.698917349E12,1.698997762E12,1.699150984E12,1.699262496E12,1.699305768E12,1.699438551E12,1.699446045E12,1.699604768E12,1.700199252E12,1.700859958E12,1.701171354E12,1.701267637E12,1.701326071E12,1.701354864E12,1.701929561E12,1.702138481E12,1.7021829E12,1.702243589E12,1.702420758E12,1.702570138E12,1.702571214E12,1.702571544E12,1.702571594E12,1.702571701E12,1.702572808E12,1.702572965E12,1.702573167E12,1.702573807E12,1.702574918E12,1.70257599E12,1.702575997E12,1.702576457E12,1.702576787E12,1.702577473E12,1.702578144E12,1.702578409E12,1.702580058E12,1.702580113E12,1.702580612E12,1.702580718E12,1.702583537E12,1.702584982E12,1.702586815E12,1.702588112E12,1.702589487E12,1.702589764E12,1.702595914E12,1.702600349E12,1.702602705E12,1.702604071E12,1.702604589E12,1.702606606E12,1.702607515E12,1.702607605E12,1.702607637E12,1.702610308E12,1.702611481E12,1.702611619E12,1.702612913E12,1.702619082E12,1.702619977E12,1.702620397E12,1.702621863E12,1.702622691E12,1.702625337E12,1.702625774E12,1.702626615E12,1.702627192E12,1.702631685E12,1.702631707E12,1.702634629E12,1.702640242E12,1.702641596E12,1.702649757E12,1.702650222E12,1.702654911E12,1.702656319E12,1.702657855E12,1.702661518E12,1.702663018E12,1.702664117E12,1.702664389E12,1.702664766E12,1.702666833E12,1.702674677E12,1.702683154E12,1.702691024E12,1.702698043E12,1.702702762E12,1.702704242E12,1.702737467E12,1.702739129E12,1.702741355E12,1.702742956E12,1.70274538E12,1.702758883E12,1.702763061E12,1.702770642E12,1.702771579E12

The plot displays the cumulative growth of stars, reflecting how interest in the Kandy library has evolved. Key points of significant increase can often be associated with major announcements or events related to the library.

To better understand how user interest in our library evolves over time, we’ll animate this chart using the Kotlin Jupyter API. This dynamic visualization will help us see how engagement patterns shift and grow, providing deeper insights than a static chart could offer.
We'll start by creating a function that builds a star history chart for the first `n` star(s). 

In [27]:
fun kandyStarHistoryPlot(n: Int) = kandyStarHistory.plot {
    line {
        x(starredAt.take(n)) {
            axis {
                name = "date"
                breaks(format = "%b, %Y")
            }
        }
        y(starsCount.take(n)) {
            axis.name = "GitHub stars"
        }
    }
    layout {
        title = "Kandy GitHub star history"
        size = 800 to 500
    }
}

Then, we'll use the `ANIMATE()` function to update the cell output for a given set of frames. Each frame will be a star history plot, starting with one star and incrementing by one star each frame until we reach the maximum number of stars.

In [28]:
ANIMATE(100.milliseconds, kandyStarHistory.rowsCount()) { frameID ->
    // frame with `frameID` contsins plot with `frameID + 1` stars
    kandyStarHistoryPlot(frameID + 1)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="RcSAka"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Kandy GitHub star history"
},
"mapping":{
},
"data":{
"starredAt":[1.657752376E12,1.66765807E12,1.667673757E12,1.670632259E12,1.67375561E12,1.673772631E12,1.674387545E12,1.674750529E12,1.674976453E12,1.675350115E12,1.675408744E12,1.675588285E12,1.675674089E12,1.675680685E12,1.675715159E12,1.67573036E12,1.675748333E12,1.6757488E12,1.675749329E12,1.675751863E12,1.675758317E12,1.67578179E12,1.675861417E12,1.675861829E12,1.675913952E12,1.675922947E12,1.675948684E12,1.675958048E12,1.676047476E12,1.676048333E12,1.676391105E12,1.676496362E12,1.676597958E12,1.676948714E12,1.677378223E12,1.67897314E12,1.679221344E12,1.679700779E12,1.6799184E12,1.68036127E12,1.681047606E12,1.681327305E12,1.6814385E12,1.68143885E12,1.681590162E12,1.681652693E12,1.681824155E12,1.682201702E12,1.682251358E12,1.682276556E12,1.68293626E12,1.683033915E12,1.683047937E12,1.683885575E12,1.683985043E12,1.685620132E12,1.68585262E12,1.686015721E12,1.686098215E12,1.68650696E12,1.68738686E12,1.687491503E12,1.687681586E12,1.688399934E12,1.688521939E12,1.688582287E12,1.688646578E12,1.688669174E12,1.689026092E12,1.689282982E12,1.689341997E12,1.689382129E12,1.6899506E12,1.689953276E12,1.689955396E12,1.689957355E12,1.689969537E12,1.689994317E12,1.690021986E12,1.690022974E12,1.690049489E12,1.69016283E12,1.690162892E12,1.690563562E12,1.690741188E12,1.690779996E12,1.691125019E12,1.691150646E12,1.691652159E12,1.691916388E12,1.692150284E12,1.692236017E12,1.692947778E12,1.692957081E12,1.693225707E12,1.693399671E12,1.693665307E12,1.694511733E12,1.694824385E12,1.694837018E12,1.694944123E12,1.696683456E12,1.696907638E12,1.697100579E12,1.697811722E12,1.698252394E12,1.698262479E12,1.698262841E12,1.698305842E12,1.698309951E12,1.698324538E12,1.698357092E12,1.698366E12,1.698368801E12,1.698374102E12,1.698377327E12,1.698378851E12,1.698378863E12,1.698387957E12,1.698388163E12,1.698388984E12,1.698392645E12,1.698394632E12,1.698397319E12,1.698397404E12,1.698401725E12,1.698402252E12,1.698411376E12,1.698416804E12,1.698424836E12,1.698670733E12,1.698677571E12,1.698683248E12,1.698743417E12,1.698744643E12,1.698876002E12,1.698917349E12,1.698997762E12,1.699150984E12,1.699262496E12,1.699305768E12,1.699438551E12,1.699446045E12,1.699604768E12,1.700199252E12,1.700859958E12,1.701171354E12,1.701267637E12,1.701326071E12,1.701354864E12,1.701929561E12,1.702138481E12,1.7021829E12,1.702243589E12,1.702420758E12,1.702570138E12,1.702571214E12,1.702571544E12,1.702571594E12,1.702571701E12,1.702572808E12,1.702572965E12,1.702573167E12,1.702573807E12,1.702574918E12,1.70257599E12,1.702575997E12,1.702576457E12,1.702576787E12,1.702577473E12,1.702578144E12,1.702578409E12,1.702580058E12,1.702580113E12,1.702580612E12,1.702580718E12,1.702583537E12,1.702584982E12,1.702586815E12,1.702588112E12,1.702589487E12,1.702589764E12,1.702595914E12,1.702600349E12,1.702602705E12,1.702604071E12,1.702604589E12,1.702606606E12,1.702607515E12,1.702607605E12,1.702607637E12,1.702610308E12,1.702611481E12,1.702611619E12,1.702612913E12,1.702619082E12,1.702619977E12,1.702620397E12,1.702621863E12,1.702622691E12,1.702625337E12,1.702625774E12,1.702626615E12,1.702627192E12,1.702631685E12,1.702631707E12,1.702634629E12,1.702640242E12,1.702641596E12,1.702649757E12,1.702650222E12,1.702654911E12,1.702656319E12,1.702657855E12,1.702661518E12,1.702663018E12,1.702664117E12,1.702664389E12,1.702664766E12,1.702666833E12,1.702674677E12,1.702683154E12,1.702691024E12,1.702698043E12,1.702702762E12,1.702704242E12,1.702737467E12,1.702739129E12,1.702741355E12,1.702742956E12,1.70274538E12,1.702758883E12,1.702763061E12,1.702770642E12,1.702771579E12

![kandy_star_history](https://github.com/user-attachments/assets/ca6e48a4-8411-4592-85da-587e26ab8bf8)

## Analyzing key events

We'll look at how different events influenced the growth of stars. We’ll add mark lines with the most important events related to Kandy, such as the [Kotlin Notebook video](https://youtu.be/m4Cqz2_P9rI?si=zn9z0vrSTu02Ah_q), the [Kandy introductory post](https://blog.jetbrains.com/kotlin/2023/12/kandy-the-new-kotlin-plotting-library-by-jetbrains/), the [Plotting Financial Data in Kotlin with Kandy](https://medium.com/@andrejkingsley/plotting-financial-data-in-kotlin-with-kandy-66757aef05ef) post, and KotlinConf 2024. Such analysis helps to identify what drives interest and engagement with the project.

We'll look at events starting from October 2023, which was when we initiated our marketing activities:

In [29]:
val starHistoryFiltered = kandyStarHistory.filter { starredAt >= LocalDateTime(2023, 10, 1, 0, 0, 0, 0) }

Then we’ll add mark lines with the events:

In [30]:
val ktnbYTVideodate = LocalDate(2023, 10, 25)
val kandyIntroductoryPostDate = LocalDate(2023, 12, 14)
val kandyFinancialPostDate = LocalDate(2024, 4, 9)
val kotlinConf24Date = LocalDate(2024, 5, 22)

val kandyEvents = listOf(
    "Kotlin Notebook\nYouTube video",
    "Kandy Introduction\nKotlin Blog post",
    "Financial Plotting\nMedium post",
    "KotlinConf 2024"
)

val kandyEventsDates = listOf(ktnbYTVideodate, kandyIntroductoryPostDate, kandyFinancialPostDate, kotlinConf24Date)

In [31]:
// Create a custom palette for the event color scale
val eventColors = listOf(
    Color.hex("#1f77b4"),
    Color.hex("#ff7f0e"),
    Color.hex("#d62728"),
    Color.hex("#2ca02c"),
)

In [32]:
starHistoryFiltered.plot {
    // add vertical marklines with event dates
    vLine {
        color(kandyEvents, "event") { scale = categorical(eventColors, kandyEvents) }
        xIntercept(kandyEventsDates)
        width = 1.5
        alpha = 0.9
    }
    line {
        x(starredAt) {axis.name = "date" }
        y(starsCount) { axis.name = "GitHub stars" }
    }
    layout {
        title = "Kandy GitHub star history & key events"
        size = 800 to 500
        style {
            legend.position = LegendPosition.Bottom
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Jv9f97"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Kandy GitHub star history & key events"
},
"mapping":{
},
"data":{
"starredAt":[1.696683456E12,1.696907638E12,1.697100579E12,1.697811722E12,1.698252394E12,1.698262479E12,1.698262841E12,1.698305842E12,1.698309951E12,1.698324538E12,1.698357092E12,1.698366E12,1.698368801E12,1.698374102E12,1.698377327E12,1.698378851E12,1.698378863E12,1.698387957E12,1.698388163E12,1.698388984E12,1.698392645E12,1.698394632E12,1.698397319E12,1.698397404E12,1.698401725E12,1.698402252E12,1.698411376E12,1.698416804E12,1.698424836E12,1.698670733E12,1.698677571E12,1.698683248E12,1.698743417E12,1.698744643E12,1.698876002E12,1.698917349E12,1.698997762E12,1.699150984E12,1.699262496E12,1.699305768E12,1.699438551E12,1.699446045E12,1.699604768E12,1.700199252E12,1.700859958E12,1.701171354E12,1.701267637E12,1.701326071E12,1.701354864E12,1.701929561E12,1.702138481E12,1.7021829E12,1.702243589E12,1.702420758E12,1.702570138E12,1.702571214E12,1.702571544E12,1.702571594E12,1.702571701E12,1.702572808E12,1.702572965E12,1.702573167E12,1.702573807E12,1.702574918E12,1.70257599E12,1.702575997E12,1.702576457E12,1.702576787E12,1.702577473E12,1.702578144E12,1.702578409E12,1.702580058E12,1.702580113E12,1.702580612E12,1.702580718E12,1.702583537E12,1.702584982E12,1.702586815E12,1.702588112E12,1.702589487E12,1.702589764E12,1.702595914E12,1.702600349E12,1.702602705E12,1.702604071E12,1.702604589E12,1.702606606E12,1.702607515E12,1.702607605E12,1.702607637E12,1.702610308E12,1.702611481E12,1.702611619E12,1.702612913E12,1.702619082E12,1.702619977E12,1.702620397E12,1.702621863E12,1.702622691E12,1.702625337E12,1.702625774E12,1.702626615E12,1.702627192E12,1.702631685E12,1.702631707E12,1.702634629E12,1.702640242E12,1.702641596E12,1.702649757E12,1.702650222E12,1.702654911E12,1.702656319E12,1.702657855E12,1.702661518E12,1.702663018E12,1.702664117E12,1.702664389E12,1.702664766E12,1.702666833E12,1.702674677E12,1.702683154E12,1.702691024E12,1.702698043E12,1.702702762E12,1.702704242E12,1.702737467E12,1.702739129E12,1.702741355E12,1.702742956E12,1.70274538E12,1.702758883E12,1.702763061E12,1.702770642E12,1.702771579E12,1.702785338E12,1.702800455E12,1.702801848E12,1.702813892E12,1.702816234E12,1.702816558E12,1.702822215E12,1.702844031E12,1.702848572E12,1.702861471E12,1.702863725E12,1.702871585E12,1.702880011E12,1.702883828E12,1.702887556E12,1.702901677E12,1.702910349E12,1.702917798E12,1.702918623E12,1.702925995E12,1.702929174E12,1.702958116E12,1.702965525E12,1.702975583E12,1.703011221E12,1.703013018E12,1.703015166E12,1.703066E12,1.703069267E12,1.703127523E12,1.703136292E12,1.703136707E12,1.7031566E12,1.703180214E12,1.70325851E12,1.703273947E12,1.703281274E12,1.703296585E12,1.703337471E12,1.703465257E12,1.703486869E12,1.703501812E12,1.703524697E12,1.703609785E12,1.703625809E12,1.703659087E12,1.703679335E12,1.703719402E12,1.703811228E12,1.703813964E12,1.703814968E12,1.703824973E12,1.703864628E12,1.703931867E12,1.704024805E12,1.704025025E12,1.704025106E12,1.704025604E12,1.704025886E12,1.704028218E12,1.704029776E12,1.704030532E12,1.704031207E12,1.704032551E12,1.70403284E12,1.704032974E12,1.704046582E12,1.704050717E12,1.704050768E12,1.704061175E12,1.704070151E12,1.704089144E12,1.704111467E12,1.704113244E12,1.704142829E12,1.704144104E12,1.704164862E12,1.704185281E12,1.704185988E12,1.70418891E12,1.704215492E12,1.70427231E12,1.704327145E12,1.704357069E12,1.704387008E12,1.704431414E12,1.704434084E12,1.704435671E12,1.704436987E12,1.704438149E12,1.70444131E12,1.704447339E12,1.704447594E12,1.704449893E12,1.704450221E12,1.704450255E12,1.704466678E12,1.704470189E12,1.704470878E12,1.7044

This plot shows the number of stars Kandy received each month, with different colors representing key events that influenced these numbers. For example, the introductory post and other significant updates coincide with noticeable increases in stars, highlighting the influence of these activities on community engagement.

## Monthly star growth analysis

To analyze the monthly growth of stars, we will create a bar chart to visually display the changes in the number of stars received each month. This visualization will help us identify key growth periods and evaluate the effectiveness of our marketing strategies.
We'll start by adding a "month" column to our DataFrame, which converts the `LocalDate`/`LocalDateTime` to a month and four-figure year format.
Here are the simple extensions that perform this conversion:

In [33]:
fun LocalDate.toMonthOfYear(): String = "$month, $year"
fun LocalDateTime.toMonthOfYear(): String = "$month, $year"

Now, we'll add the "month" column to our DataFrame:

In [34]:
val starHistoryWithMonth = starHistoryFiltered.add("month") {
    starredAt.toMonthOfYear()
}
starHistoryWithMonth

starredAt,login,starsCount,month
2023-10-07T12:57:36,vaabr,102,"OCTOBER, 2023"
2023-10-10T03:13:58,milksense,103,"OCTOBER, 2023"
2023-10-12T08:49:39,citywalki,104,"OCTOBER, 2023"
2023-10-20T14:22:02,yeradis,105,"OCTOBER, 2023"
2023-10-25T16:46:34,alexwhb,106,"OCTOBER, 2023"
2023-10-25T19:34:39,alibagherifam,107,"OCTOBER, 2023"
2023-10-25T19:40:41,sndpg,108,"OCTOBER, 2023"
2023-10-26T07:37:22,rmarquis,109,"OCTOBER, 2023"
2023-10-26T08:45:51,calt-laboratory,110,"OCTOBER, 2023"
2023-10-26T12:48:58,michaelbgd,111,"OCTOBER, 2023"


Next, we’ll group the DataFrame by the "month" column and count the number of stars in each group. 

In [35]:
val starsCountMonthly = starHistoryWithMonth.groupBy { month }.count()
starsCountMonthly

month,count
"OCTOBER, 2023",34
"NOVEMBER, 2023",15
"DECEMBER, 2023",155
"JANUARY, 2024",66
"FEBRUARY, 2024",34
"MARCH, 2024",13
"APRIL, 2024",47
"MAY, 2024",30
"JUNE, 2024",28
"JULY, 2024",17


Next, we'll add information about key events to the DataFrame. We'll include the events in the corresponding months and set the value to null if there were no events. First, create a DataFrame with events and their corresponding months:

In [36]:
val eventsDF = dataFrameOf("event" to kandyEvents, "month" to kandyEventsDates.map {
    it.toMonthOfYear()
})

Then, perform a left join with our main DataFrame at the month column:

In [37]:
val starsMonthlyWithEvent = starsCountMonthly.leftJoin(eventsDF) { month }
starsMonthlyWithEvent

month,count,event
"OCTOBER, 2023",34,Kotlin Notebook YouTube video
"NOVEMBER, 2023",15,null
"DECEMBER, 2023",155,Kandy Introduction Kotlin Blog post
"JANUARY, 2024",66,null
"FEBRUARY, 2024",34,null
"MARCH, 2024",13,null
"APRIL, 2024",47,Financial Plotting Medium post
"MAY, 2024",30,KotlinConf 2024
"JUNE, 2024",28,null
"JULY, 2024",17,null


Now, we can create a bar plot to visualize the distribution of new stars by month, along with the key events.

In [38]:
starsMonthlyWithEvent.plot {
    bars {
        x(month)
        y(count)
        alpha = 0.8
        fillColor(event) { scale = categorical(eventColors, kandyEvents) }
    }
    // add horizontal markline with median of monthly count
    hLine {
        val medianMonthly = count.median()
        yIntercept.constant(medianMonthly)
        type = LineType.DASHED
        color = Color.hex("#4b0082")
        width = 2.0
    }
    layout {
        title = "Kandy GitHub star history (montly count)"
        size = 800 to 500
        style {
            legend.position = LegendPosition.Bottom
            xAxis.text { angle = 30.0 }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SGeJqR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Kandy GitHub star history (montly count)"
},
"mapping":{
},
"data":{
"month":["OCTOBER, 2023","NOVEMBER, 2023","DECEMBER, 2023","JANUARY, 2024","FEBRUARY, 2024","MARCH, 2024","APRIL, 2024","MAY, 2024","JUNE, 2024","JULY, 2024","AUGUST, 2024","SEPTEMBER, 2024","OCTOBER, 2024","NOVEMBER, 2024","DECEMBER, 2024","JANUARY, 2025","FEBRUARY, 2025","MARCH, 2025","APRIL, 2025","MAY, 2025","JUNE, 2025"],
"count":[34.0,15.0,155.0,66.0,34.0,13.0,47.0,30.0,28.0,17.0,25.0,10.0,7.0,20.0,15.0,5.0,8.0,12.0,14.0,14.0,11.0],
"event":["Kotlin Notebook\nYouTube video",null,"Kandy Introduction\nKotlin Blog post",null,null,null,"Financial Plotting\nMedium post","KotlinConf 2024",null,null,null,null,null,null,null,null,null,null,null,null,null]
},
"ggsize":{
"width":800.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#1f77b4","#ff7f0e","#d62728","#2ca02c"],
"limits":["Kotlin Notebook\nYouTube video","Kandy Introduction\nKotlin Blog post","Financial Plotting\nMedium post","KotlinConf 2024"]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"count",
"fill":"event"
},
"stat":"identity",
"sampling":"none",
"alpha":0.8,
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":15.0,
"color":"#4b0082",
"size":2.0,
"linetype":"dashed",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
}],
"theme":{
"axis_text_x":{
"angle":30.0,
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"bottom"
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"month"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"event"
}]
},
"spec_id":"2051"
};
 var containerDiv = document.getElementById("SGeJqR");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 500.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OCTOBER, 2023 
 
 
 
 
 
 
 
 
 NOVEMBER, 2023 
 
 
 
 
 
 
 
 
 DECEMBER, 2023 
 
 
 
 
 
 
 
 
 JANUARY, 2024 
 
 
 
 
 
 
 
 
 FEBRUARY, 2024 
 
 
 
 
 
 
 
 
 MARCH, 2024 
 
 
 
 
 
 
 
 
 APRIL, 2024 
 
 
 
 
 
 
 
 
 MAY, 2024 
 
 
 
 
 
 
 
 
 JUNE, 2024 
 
 
 
 
 
 
 
 
 JULY, 2024 
 
 
 
 
 
 
 
 
 AUGUST, 2024 
 
 
 
 
 
 
 
 
 SEPTEMBER, 2024 
 
 
 
 
 
 
 
 
 OCTOBER, 2024 
 
 
 
 
 
 
 
 
 NOVEMBER, 2024 
 
 
 
 
 
 
 
 
 DECEMBER, 2024 
 
 
 
 
 
 
 
 
 JANUARY, 2025 
 
 
 
 
 
 
 
 
 FEBRUARY, 2025 
 
 
 
 
 
 
 
 
 MARCH, 2025 
 
 
 
 
 
 
 
 
 APRIL, 2025 
 
 
 
 
 
 
 
 
 MAY, 2025 
 
 
 
 
 
 
 
 
 JUNE, 2025 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kandy GitHub star history (montly count) 
 
 
 
 
 count 
 
 
 
 
 month 
 
 
 
 
 
 
 
 
 event 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Notebook 
 
 
 YouTube video 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kandy Introduction 
 
 
 Kotlin Blog post 
 

This plot shows the monthly distribution of stars, with bars representing the number of stars each month. The colors of the bars indicate key events, providing a clear visualization of how these events impacted the star counts. The dashed horizontal line represents the median star count per month.

Unlike the overall star history chart, which shows cumulative growth, the monthly statistics plot helps you pinpoint the exact timing and impact of key events. By creating similar plots for your own projects, you can better understand the effectiveness of your promotional efforts, identify seasonal patterns, and plan future activities more effectively.

### Understanding your audience

Understanding the top programming languages of your stargazers can provide insights into your audience. With this in mind, we’ll use the GitHub REST API to find out the most popular languages among Kandy stargazers and visualize this data as a pie chart.

Let's write a function that requests user repositories:

In [39]:
import io.ktor.http.*

fun getUserRepos(login: String): AnyFrame {
    return DataFrame.readJsonStr(http.get("https://api.github.com/users/$login/repos") {
        // Set authorization header with token
        bearerAuth(token)
        // Add GitHub API custom "accept" header
        header(HttpHeaders.Accept, "application/vnd.github.v3+json")
    }.deserializeJson().jsonString)
}

Next, we'll test this function on our sample repositories:

In [40]:
val myRepos = getUserRepos("Kotlin")
myRepos

<iframe onload="o_resize_iframe_out_17()" style="width:100%;" class="result_container" id="iframe_out_17" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-385875932"></table>

<p class="dataframe_description">... showing only top 20 of 30 rows</p><p class="dataframe_description">DataFrame: rowsCount = 30, columnsCount = 80</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.l

Each column in this DataFrame corresponds to a repository and contains different information about that repository. We are interested in the `language` column. We can count the most frequent language using the `.valueCount()` method, where the first entry represents the most popular language:

In [41]:
val myLanguagesCounts = myRepos.language.valueCounts(dropNA = false) // Don't drop nulls
myLanguagesCounts

language,count
Kotlin,24
null,4
Jupyter Notebook,1
Markdown,1


Because the rows are sorted by count by default, identifying the most popular language is straightforward – it’s the first one.

In [42]:
myLanguagesCounts.language.first()

Kotlin

To generalize this process, we'll write an extension function for a DataFrame obtained from a user repositories request. This extension function will retrieve the most popular language (returning `null` if the account is private, has no repositories, or lacks sufficient information).

In [43]:
fun AnyFrame.getTopLanguage(): String? {
    //  Handle non-default response bodies (private account, no repositories, etc.)
    if (!containsColumn("language")) return null
    return castTo(myRepos).language
        .valueCounts(dropNA = false)
        .castTo(myLanguagesCounts)
        .language.let { languages ->
            val first = languages.firstOrNull()
            //  Try to pick the second value if the first one is null
            if (first == null && languages.size() >= 2) {
                languages[1]
            } else first
        }
}

Now, let's retrieve the most popular languages for all stargazers. Note, that this process might take some time to execute:

In [44]:
val stargazersLanguages = kandyStarHistory.select {
    login and login.map { login -> getUserRepos(login).getTopLanguage() }.named("language")
}
stargazersLanguages

java.lang.IllegalArgumentException: Column has schema:
 topics: List<Nothing>
 that differs from target schema:
 topics: List<String>

Next, we'll count the occurrences of each language:

In [35]:
val languageCounts = stargazersLanguages.language.valueCounts() // Drops null by default
languageCounts

language,count
Kotlin,221
Java,120
JavaScript,35
Python,27
Rust,11
Dart,10
HTML,10
TypeScript,9
Swift,9
C++,8


Finally, let's plot these counts as a pie chart. We'll take the first seven most popular languages and group the remaining ones into an "other" category:

In [36]:
languageCounts.let {
    val takeFirst = 7
    it.take(takeFirst).concat(
        dataFrameOf("language" to listOf("other"), "count" to listOf(it.drop(takeFirst).sum {count}))
    )
}.plot {
    pie {
        slice("count")
        fillColor("language")
        size = 25.0
        hole = 0.3
    }
    layout {
        title = "Kandy stargazers' most popular languages"
        style(Style.Void)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zchvE8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Kandy stargazers' most popular languages"
},
"mapping":{
},
"data":{
"count":[221.0,120.0,35.0,27.0,11.0,10.0,10.0,75.0],
"language":["Kotlin","Java","JavaScript","Python","Rust","Dart","HTML","other"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"hole":0.3,
"mapping":{
"slice":"count",
"fill":"language"
},
"stat":"identity",
"size":25.0,
"sampling":"none",
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("zchvE8");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kandy stargazers' most popular languages 
 
 
 
 
 
 
 
 
 language 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 JavaScript 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Python 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rust 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Dart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HTML 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 other

The pie chart shows that Kotlin is the most popular language among Kandy stargazers, confirming our primary audience as Kotlin developers. The presence of Java suggests potential for further engagement with related ecosystems. The inclusion of less common languages highlights the diversity of our user base, which is important for understanding different use cases and potential feature requests. 
These insights can help tailor your project's documentation, tutorials, and marketing efforts to better serve and expand your audience.

## Comparing star growth: Kandy vs. Kotlin DataFrame

Comparing star data across different projects can provide valuable insights into their popularity and user engagement. Here, we'll look at the growth of stars for Kandy alongside Kotlin DataFrame. These two projects, launched within a year of each other, target the same audience of Kotlin developers.

To ensure a fair comparison, we'll use the introduction post date as the starting point for both libraries and examine the six months that followed. This way, we can see how each project grew over the same timeframe, giving us a clearer picture of their growth patterns.

In [37]:
val repoDataframe = "dataframe"
// Use the already written methods to get star history for DataFrame
val dataFrameStarHistory = fetchStarHistory(ownerKotlin, repoDataframe, token).processStargazers()

In [38]:
val dataFrameIntroductoryPostDate = LocalDate(2022, 6, 30)

In [39]:
// Function that will slightly transform the dataframe with star history for a given library: 
// 1) Take a period of six months after the introduction post date; 
// 2) Add a column "daysAfterPost" with the number of days after the post date; 
// 3) Take the maximum number of stars for the day; 
// 4) Add a column "library" corresponding to the name of the library.
fun AnyFrame.proccessAfterPostPeriod(introductionPostDate: LocalDate, library: String): AnyFrame {
    // Six-month period after `introductionPostDate`
    val period = (introductionPostDate - DatePeriod(days = 1))..(introductionPostDate + DatePeriod(months = 6))
    return castTo(kandyStarHistory)
        // Only take stars placed during that period
        .filter { starredAt.date in period }
        // Add daysAfterPost column with number of days after post
        .add("daysAfterPost") {
            introductionPostDate.daysUntil(starredAt.date)
        }
        // Group by number of days and take the max value of `starsCount` for each group
        .groupBy("daysAfterPost").max { starsCount }
        // Add a column with library name
        .add("library") { library }
}

In [40]:
// Count six-month history for both libraries and concatenate them into one DataFrame
val kandyAndDataFrameStarHistory = kandyStarHistory
    .proccessAfterPostPeriod(kandyIntroductoryPostDate, "Kandy")
    .concat(
        dataFrameStarHistory.proccessAfterPostPeriod(dataFrameIntroductoryPostDate, "DataFrame")
    )
kandyAndDataFrameStarHistory

daysAfterPost,starsCount,library
0,185,Kandy
1,225,Kandy
2,237,Kandy
3,248,Kandy
4,260,Kandy
5,266,Kandy
6,268,Kandy
7,273,Kandy
8,276,Kandy
9,278,Kandy


Next, we'll visualize the comparison:

In [41]:
kandyAndDataFrameStarHistory.plot {
    line {
        x(daysAfterPost) {
            axis {
                name = "days after post"
            }
        }
        y(starsCount) {
            axis.name = "GitHub stars"
        }
        color(library)
    }
    layout {
        title = "Kandy vs. DataFrame GitHub stars history\nwithin 6 months after the introductory post"
        size = 800 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hYbfDY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Kandy vs. DataFrame GitHub stars history\nwithin 6 months after the introductory post"
},
"mapping":{
},
"data":{
"daysAfterPost":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,28.0,29.0,31.0,33.0,34.0,35.0,36.0,37.0,39.0,40.0,41.0,43.0,46.0,48.0,49.0,50.0,51.0,53.0,54.0,56.0,59.0,60.0,61.0,63.0,64.0,68.0,69.0,70.0,72.0,73.0,75.0,76.0,77.0,78.0,80.0,82.0,84.0,86.0,93.0,97.0,99.0,105.0,106.0,109.0,110.0,111.0,116.0,117.0,118.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,129.0,130.0,131.0,132.0,134.0,139.0,141.0,144.0,145.0,146.0,148.0,149.0,152.0,153.0,155.0,159.0,160.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,171.0,172.0,173.0,174.0,176.0,177.0,178.0,179.0,181.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,12.0,16.0,17.0,18.0,20.0,22.0,23.0,26.0,27.0,28.0,31.0,32.0,34.0,39.0,43.0,45.0,47.0,48.0,51.0,52.0,53.0,54.0,55.0,57.0,58.0,60.0,61.0,62.0,63.0,65.0,67.0,68.0,73.0,74.0,76.0,80.0,85.0,89.0,92.0,93.0,95.0,98.0,104.0,105.0,106.0,110.0,112.0,113.0,116.0,117.0,126.0,127.0,128.0,132.0,141.0,147.0,149.0,153.0,157.0,159.0,168.0,171.0,173.0,181.0,182.0],
"library":["Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","Kandy","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame","DataFrame"],
"starsCount":[185.0,225.0,237.0,248.0,260.0,266.0,268.0,273.0,276.0,278.0,282.0,284.0,287.0,292.0,293.0,309.0,316.0,321.0,322.0,325.0,343.0,346.0,348.0,350.0,351.0,353.0,355.0,356.0,357.0,359.0,360.0,363.0,365.0,367.0,369.0,372.0,375.0,376.0,378.0,379.0,380.0,381.0,384.0,385.0,386.0,387.0,389.0,391.0,394.0,400.0,401.0,403.0,404.0,405.0,406.0,408.0,409.0,412.0,413.0,414.0,415.0,416.0,418.0,419.0,421.0,422.0,424.0,425.0,426.0,427.0,429.0,432.0,438.0,442.0,446.0,450.0,453.0,455.0,458.0,460.0,461.0,462.0,466.0,468.0,469.0,470.0,472.0,474.0,475.0,476.0,477.0,478.0,479.0,480.0,481.0,482.0,484.0,486.0,487.0,488.0,495.0,499.0,500.0,

From the initial observation, we can see that before the introduction post, both Kandy and Kotlin DataFrame had similar star counts. However, immediately after the post, Kandy showed a significantly higher growth rate, achieving nearly twice the number of stars as DataFrame within the first six months.

This difference suggests several things. Firstly, it shows the growing interest in Kotlin for data projects. The period of time that elapsed from the initial DataFrame post and the Kandy post was about a year and a half. While DataFrame helped establish a community of Kotlin data enthusiasts, Kandy attracted a new audience interested in visualization.

Additionally, Kandy had more intense promotional activities within the six months following its first post, which likely contributed to its rapid growth.

### Shared stargazers

It’s also interesting to see how many users starred both Kandy and DataFrame. We hypothesize that there will be a significant overlap since both libraries serve the same community of Kotlin developers. Here's how we can analyze this and get the relevant data:

In [42]:
// inner join star history dataframes of repositories by login,
// getting a dataframe with all common stargazers, taking its size to get a number of them
val commonStargazers = kandyStarHistory.innerJoin(dataFrameStarHistory) { login }.rowsCount()
val kandyTotalStargazers = kandyStarHistory.rowsCount()
val kandyOnlyStargazers = kandyTotalStargazers - commonStargazers
val dataFrameTotalStargazers = dataFrameStarHistory.rowsCount()
val dataFrameOnlyStargazers = dataFrameTotalStargazers - commonStargazers

Plot this data as a pie chart:

In [43]:
plot {
    pie {
        slice(listOf(commonStargazers, kandyOnlyStargazers, dataFrameOnlyStargazers))
        fillColor(listOf("Common", "Kandy only", "DataFrame only")) {
            scale = categorical(
                "Common" to Color.hex("#4A90E2"),
                "Kandy only" to Color.hex("#F5A623"),
                "DataFrame only" to Color.hex("#7ED321"),
            )
            legend.name = ""
        }
        size = 25.0
    }
    layout {
        title = "Kandy & DataFrame stargazers ratio"
        style(Style.Void)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="5YGUS7"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Kandy & DataFrame stargazers ratio"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"values":["#4A90E2","#F5A623","#7ED321"],
"name":"",
"limits":["Common","Kandy only","DataFrame only"]
}],
"layers":[{
"mapping":{
"slice":"slice",
"fill":"fill"
},
"stat":"identity",
"data":{
"slice":[150.0,412.0,641.0],
"fill":["Common","Kandy only","DataFrame only"]
},
"size":25.0,
"sampling":"none",
"position":"identity",
"geom":"pie"
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("5YGUS7");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kandy & DataFrame stargazers ratio 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Common 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kandy only 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DataFrame only

The analysis shows that a large number of stargazers are unique to DataFrame, with fewer users starring both DataFrame and Kandy. Specifically, the share of DataFrame stargazers who also starred Kandy is quite small. This is probably because many users use DataFrame for data tasks that don’t involve visualization, making Kandy less relevant to them.

Interestingly, only about a quarter of Kandy stargazers have also starred DataFrame. This suggests that Kandy has attracted a new audience mainly interested in plotting, rather than data processing. This reveals a great opportunity to promote how both libraries can work together.

Using Kandy for visualization and DataFrame for data processing allows users to benefit from the strengths of both libraries. This combination, as we've shown in this post, can help create powerful and comprehensive data analysis solutions. By highlighting this synergy, we can encourage more users to explore how these tools can complement each other and enhance their data projects.

## Conclusion 

In this post, we demonstrated how to use Kotlin DataFrame and Kandy to analyze and visualize the star history of GitHub repositories. By leveraging the GitHub GraphQL API, we fetched data about stargazers, processed it to extract valuable insights, and created visualizations to better understand the popularity trends and audience of the project. 

We analyzed the monthly distribution of stars, highlighted key events that influenced the growth of the repository, explored the top programming languages of stargazers, compared our growth with the DataFrame library, and examined the overlap in user bases between the two libraries. These insights can help you tailor your project's documentation, tutorials, and marketing efforts to better serve and expand your audience.

### What's next?

Now it's your turn! Apply these techniques to your own repositories, analyze their star history, and create your own visualizations within Kotlin Notebook.

We’d love to see your results and hear your feedback. Join us in the #datascience channel on Kotlin Slack, or reach out via GitHub issues for Kandy or DataFrame.

If you find our repositories useful, we’d really appreciate it if you’d star them. Your support helps us improve and develop these tools further.